## Using class as decorator

In [1]:
class my_decorator(object):
    def __init__(self, func):
        self._func = func
    def __call__(self, *args, **kwargs):
        print('Called {}(*({}), **({}))'.format(self._func.__name__, args, kwargs))
        print('Self is {}'.format(self))
        return self._func(*args, **kwargs)
    def __str__(self):
        return 'Decorator for {}'.format(self._func.__name__)

In [2]:
@my_decorator
def f(x, y):
    return x ** y

In [3]:
f(5, 6)

Called f(*((5, 6)), **({}))
Self is Decorator for f


15625

In [4]:
class Calculator(object):
    def __init__(self, init_value):
        self.coef = init_value
    @my_decorator
    def calc(self, x, y):
        return x * y * self.coef

In [6]:
c = Calculator(10)
c.calc(2, 3)

Called calc(*((2, 3)), **({}))
Self is Decorator for calc


TypeError: calc() missing 1 required positional argument: 'y'

## Decorator as function for class methods

They were much better then class decorator.
Причина в том, что при реализации декоратора как класса при обращении к задекорированному методу вызывается метод __call__ декоратора, которому передается в качестве self собственно сам декоратор. Поэтому задекорированный метод не может получить ссылку на свой родной объект. Таким образом реалзиация декоратора в виде функции более предпочтительна.

In [7]:
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print('Called {} with args: {} and kwargs: {}'.format(func.__name__, args, kwargs))
        return func(*args, **kwargs)
    return wrapper

In [9]:
@my_decorator
def func(x, y):
    return x ** y

In [10]:
func(5, 7)

Called func with args: (5, 7) and kwargs: {}


78125

In [12]:
class C:
    def __init__(self, coef):
        self.coef = coef
    @my_decorator
    def calc(self, x, y):
        return self.coef * x * y

In [13]:
c = C(3)
c.calc(12, 17)

Called calc with args: (<__main__.C object at 0x7fbe6f77bf60>, 12, 17) and kwargs: {}


612

# Декораторы классов
Появлились в 2.6 и в 3.0. Роль частично перекликается с метаклассами. 

In [15]:
def decorator(cls):
    class Wrapper(object):
        def __init__(self, *args):
            print('Init wrapper for {} with {}'.format(cls, args))
            self.wrapped = cls(*args)
        def __getattr__(self, name):
            return getattr(self.wrapped, name)
    return Wrapper

In [16]:
@decorator
class C(object):
    def __init__(self, x, y):
        self.attr = 'spam'
        self.x = x
        self.y = y

In [18]:
c = C(6, 7)
print(c.attr)
print(type(c))

Init wrapper for <class '__main__.C'> with (6, 7)
spam
<class '__main__.decorator.<locals>.Wrapper'>


## Поддержка нескольких экземпляров
Пример некорректного декоратора

In [21]:
class Decorator:
    def __init__(self, cls):
        self.cls = cls
    def __call__(self, *args):
        self.wrapped = self.cls(*args)
        print('Wrapped object is {}'.format(self.wrapped))
        return self
    def __getattr__(self, attr):
        return getattr(self.wrapped, attr)

In [22]:
@Decorator
class C:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __str__(self):
        return 'x={}, y={}'.format(self.x, self.y)

In [23]:
x = C(1, 2)

Wrapped object is x=1, y=2


In [24]:
y = C(2, 3)

Wrapped object is x=2, y=3


In [28]:
x.wrapped is y.wrapped  # wrapped rewrited

True

Возможные решения:

In [29]:
def decorator1(cls):
    class Wrapper:
        def __init__(self, *args):
            self.wrapped = cls(*args)
    return Wrapper

In [30]:
@decorator1
class C:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __str__(self):
        return 'x={}, y={}'.format(self.x, self.y)

In [33]:
x = C(3, 4)
y = C(5, 6)
x.wrapped is y.wrapped

False

In [34]:
class Wrapper(object):
    def __init__(self, wrapped):
        self.wrapped = wrapped

def decorator2(cls):
    def on_call(*args):
        return Wrapper(cls(*args))
    return on_call

In [35]:
@decorator2
class C:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __str__(self):
        return 'x={}, y={}'.format(self.x, self.y)

In [36]:
x = C(5, 6)
y = C(7, 8)
x.wrapped is y.wrapped

False

## Вложенность декораторов
Декораторы применяются в обратном порядке (самый близкий к функции будет применен самым первым)

In [40]:
def decorator_a(func):
    print('Called decorator A for {}'.format(func))
    def wrapper(*args):
        print('Wrapper A called')
        return func(*args)
    return wrapper

def decorator_b(func):
    print('Called decorator B for {}'.format(func))
    def wrapper(*args):
        print('Wrapper B called')
        return func(*args)
    return wrapper

def decorator_c(func):
    print('Called decorator C for {}'.format(func))
    def wrapper(*args):
        print('Wrapper C called')
        return func(*args)
    return wrapper

In [41]:
@decorator_a
@decorator_b
@decorator_c
def f(x):
    print('Called f({})'.format(x))
    return x

Called decorator C for <function f at 0x7fbe6ee80d08>
Called decorator B for <function decorator_c.<locals>.wrapper at 0x7fbe6ee80730>
Called decorator A for <function decorator_b.<locals>.wrapper at 0x7fbe6ee80400>


In [42]:
f(4)

Wrapper A called
Wrapper B called
Wrapper C called
Called f(4)


4

## Аргументы декораторов
Все декораторы могут принимать дополнительные аргументы, которые позволяют как-то влиять на создание и поведение обертки.

In [43]:
def decorator(*args, **kwargs):
    print('Decorator args: ', args)
    print('Decorator kwargs: ', kwargs)
    def actual_decorator(func):
        print('Called actual_decorator for {}'.format(func))
        def wrapper(*args, **kwargs):
            print('Wrapper called with {} and {}'.format(args, kwargs))
            return func(*args, **kwargs)
        return wrapper
    return actual_decorator

In [51]:
@decorator() # обязательно вызов в данном случае, декоратор будет работать некорректно
def f1(x, base=2):
    return int(x, base=base)

Decorator args:  ()
Decorator kwargs:  {}
Called actual_decorator for <function f1 at 0x7fbe6eef42f0>


In [47]:
f1(10)

Called actual_decorator for 10


<function __main__.decorator.<locals>.actual_decorator.<locals>.wrapper>

In [48]:
@decorator(10, base=12)
def f2(x, y, value=12):
    return x * y * value

Decorator args:  (10,)
Decorator kwargs:  {'base': 12}
Called actual_decorator for <function f2 at 0x7fbe6ee806a8>


In [50]:
f2(1, 2)

Wrapper called with (1, 2) and {}


24

## Управление функциями или классами

In [57]:
decorated = set()
def decorator(obj):
    decorated.add(obj)
    obj._decorated = True
    print('{} was registered'.format(obj))
    return obj

In [58]:
@decorator
def f(x):
    return x ** 2

<function f at 0x7fbe6f75aea0> was registered


In [65]:
f._decorated

True

In [61]:
@decorator
class C:
    pass

<class '__main__.C'> was registered


In [66]:
C._decorated

True

In [67]:
print(decorated)

{<function f at 0x7fbe6f75aea0>, <class '__main__.C'>}


# Программирование декораторов функций
##  Трассировка вызовов

In [70]:
class tracer:
    def __init__(self, func):
        self.calls = 0
        self.func = func
    def __call__(self, *args, **kwargs):
        self.calls += 1
        print('{fname} was called {calls} time{plural}'.format(fname=self.func.__name__, 
                                                               calls=self.calls, 
                                                               plural='' if self.calls == 1 else 's'))
        return self.func(*args, **kwargs)

In [71]:
@tracer
def spam(a, b, c):
    return a + b + c

In [74]:
spam(1, 2, 3)

spam was called 3 times


6

In [75]:
spam.calls

3

In [76]:
type(spam)

__main__.tracer

## Сохранения состояния декоратора
Вариантов несколько:

* декораторы-класс (см. выше), но тогда проблемы с декорированием методов
* глобальные переменные (с инструкцией global)
* замыкания
* атрибуты функций

### Декораторы-классы

In [78]:
class tracer:
    def __init__(self, func):
        self.calls = 0
        self.func = func
    def __call__(self, *args, **kwargs):
        self.calls += 1
        print('{fname} was called {calls} time{plural}'.format(fname=self.func.__name__, 
                                                               calls=self.calls, 
                                                               plural='' if self.calls == 1 else 's'))
        return self.func(*args, **kwargs)

In [85]:
@tracer
def spam(a, b, c=10):
    return (a + b) * c

In [81]:
@tracer
def egg(a, b):
    return a ** b

In [86]:
spam(1, 2)
spam(2, 3, c=12)
spam(3, 4)
print('Spam called: ', spam.calls)

spam was called 1 time
spam was called 2 times
spam was called 3 times
Spam called:  3


In [87]:
egg(1, 2)
egg(2, 3)
egg(2, 2)
print('Egg called:', egg.calls)

egg was called 1 time
egg was called 2 times
egg was called 3 times
Egg called: 3


### Глобальные переменные

In [93]:
calls = 0
def tracer(func):
    def wrapper(*args, **kwargs):
        global calls
        calls += 1
        print('Functions were called {calls} time{plural}'.format(calls=calls, plural='' if calls == 1 else 's'))
        return func(*args, **kwargs)
    return wrapper

In [94]:
@tracer
def spam(a, b):
    return a + b

In [95]:
@tracer
def egg(a, b):
    return a * b

In [96]:
spam(1, 2)

Functions were called 1 time


3

In [97]:
egg(1, 2)

Functions were called 2 times


2

Как вариант, можно использовать, например, глобальный словарь

In [98]:
calls = {}
def tracer(func):
    global calls
    calls.setdefault(func, 0)
    def wrapper(*args, **kwargs):
        global calls
        calls[func] += 1
        print('Functions were called {calls} time{plural}'.format(calls=calls[func], 
                                                                  plural='' if calls[func] == 1 else 's'))
        return func(*args, **kwargs)
    return wrapper

In [99]:
@tracer
def f(x):
    return x ** 2

In [101]:
@tracer
def g(x):
    return x ** 3

In [102]:
f(1)

Functions were called 1 time


1

In [103]:
g(1)

Functions were called 1 time


1

In [104]:
calls

{<function __main__.f>: 1, <function __main__.g>: 1}

### Замыкания

In [105]:
def tracer(func):
    calls = 0
    def wrapper(*args, **kwargs):
        nonlocal calls  # python 3 only
        calls += 1
        print('{f} were called {calls} time{plural}'.format(f=func.__name__, calls=calls, 
                                                            plural='' if calls == 1 else 's'))
        return func(*args, **kwargs)
    return wrapper

In [111]:
@tracer
def f(x):
    return x ** 2

In [112]:
@tracer
def g(x):
    return x ** 3

In [113]:
f(1)
f(3)
g(10)

f were called 1 time
f were called 2 times
g were called 1 time


1000

Но теперь нельзя напрямую узнать, сколько вызовов сделано каждой функцией.

### Атрибуты функции
 Работает и в 2, и в 3

In [116]:
def tracer(func):
    def wrapper(*args, **kwargs):
        wrapper.calls += 1
        print('{f} were called {calls} time{plural}'.format(f=func.__name__, 
                                                            calls=wrapper.calls, 
                                                            plural='' if wrapper.calls == 1 else 's'))
        return func(*args, **kwargs)
    wrapper.calls = 0
    return wrapper

In [117]:
@tracer
def f(x):
    return x ** 2

@tracer
def g(x):
    return x ** 3

In [118]:
f(1)
f(3)
g(10)

f were called 1 time
f were called 2 times
g were called 1 time


1000

In [119]:
f.calls, g.calls

(2, 1)

## Промахи с классами: декорирование методов

In [120]:
class tracer:
    def __init__(self, func):
        self.calls = 0
        self.func = func
    def __call__(self, *args, **kwargs):
        self.calls += 1
        print('{fname} was called {calls} time{plural}'.format(fname=self.func.__name__, 
                                                               calls=self.calls, 
                                                               plural='' if self.calls == 1 else 's'))
        return self.func(*args, **kwargs)

In [130]:
class Person(object):
    def __init__(self, name, pay):
        self.name = name
        self.pay = pay
    
    @tracer
    def give_raise(self, percent):
        self.pay *= (1.0 + percent)
    
    @tracer
    def last_name(self):
        return self.name.split()[-1]


In [131]:
bob = Person('Bob Smith', 10000)

In [132]:
bob.give_raise(0.2)

give_raise was called 1 time


TypeError: give_raise() missing 1 required positional argument: 'percent'

In [142]:
bob.last_name()

last_name was called 5 times


TypeError: last_name() missing 1 required positional argument: 'self'

Дело в том, что декоратор при вызове __call__ передает в качестве self самого себя (оно и логично - все-таки объект). Соответственно, обернутый метод становится *unbound*, т.е. вызывается отдельно от класса и не получает аргумента self. Это конечно, можно обойти (см. ниже), но надо ли оно.

In [134]:
bob.last_name(bob)

last_name was called 2 times


'Smith'

In [139]:
bob.give_raise(bob, .1)
bob.pay

give_raise was called 4 times


13310.000000000004

### Использование вложенных функций для декорирования методов

In [143]:
def tracer(func):
    calls = 0
    def on_call(*args, **kwargs):
        nonlocal calls
        calls += 1
        print('call %s to %s' % (calls, func.__name__))
        return func(*args, **kwargs)
    return on_call

In [144]:
class Person(object):
    def __init__(self, name, pay):
        self.name = name
        self.pay = pay
    
    @tracer
    def give_raise(self, percent):
        self.pay *= (1.0 + percent)
    
    @tracer
    def last_name(self):
        return self.name.split()[-1]

In [145]:
bob = Person('Bob Smith', 100000)

In [146]:
bob.give_raise(0.2)

call 1 to give_raise


In [147]:
bob.last_name()

call 1 to last_name


'Smith'

In [148]:
sam = Person('Sam Johns', 1000000)

In [149]:
sam.give_raise(0.2)

call 2 to give_raise


In [150]:
sam.give_raise

<bound method Person.on_call of <__main__.Person object at 0x7fbe6ee9c2b0>>

### Использование дескрипторов для декорирования методов

In [172]:
class tracer(object):
    def __init__(self, func):
        self.calls = 0
        self.func = func
    
    def __call__(self, *args, **kwargs):
        self.calls += 1
        print('call %s to %s' % (self.calls, self.func.__name__))
        return self.func(*args, **kwargs)
    
    def __get__(self, instance, owner):
        return wrapper(self, instance)

class wrapper:
    def __init__(self, desc, subj):
        print('Desc is {}'.format(desc))
        print('Subj is {}'.format(subj))
        self.desc = desc
        self.subj = subj
    def __call__(self, *args, **kwargs):
        print('Called wrapper __call__ with {}, {}'.format(args, kwargs))
        print('tracer for ', self.desc.func)
        return self.desc(self.subj, *args, **kwargs)

In [173]:
@tracer
def f(x):
    return x ** 2

In [174]:
f(1)

call 1 to f


1

In [175]:
f.calls

1

In [168]:
type(f)

__main__.tracer

In [176]:
class Person(object):
    def __init__(self, name, pay):
        self.name = name
        self.pay = pay
    
    @tracer
    def give_raise(self, percent):
        self.pay *= (1.0 + percent)
    
    @tracer
    def last_name(self):
        return self.name.split()[-1]

In [177]:
bob = Person('Bob Smith', 100)

In [179]:
bob.give_raise(0.2)

Desc is <__main__.tracer object at 0x7fbe6ee4bb38>
Subj is <__main__.Person object at 0x7fbe6ee4b908>
Called wrapper __call__ with (0.2,), {}
tracer for  <function Person.give_raise at 0x7fbe6ee8f620>
call 2 to give_raise


Суть в следующем. В классе tracer у нас описан метод __get__, который реализует часть протокола дескрипторов. Когда мы обращаемся к задекорированному таким образом методу *give_raise*, то происходит следующее:

* питон находит по имени в классе объект give_raise
* видит, что у него есть метод __get__
* вызывает метод __get__, который возвращает инстанс класса-обертки wrapper, которому для инициации передает два аргумента: ссылку на себя и ссылку на инстанс объекта, для которого мы вызвали метод (т.е. bob'a)
* вызывает метод __call__ класса обертки, передавая ему аргумент метода (т.е. 0.2)
* внутри метода __call__ выполняется вызов метода __call__ класса **tracer**, которому передается уже ссылка на bob'a и параметры метода.

Вот такая вот схема. 

In [180]:
class tracer(object):
    def __init__(self, func):  # On @ decorator
        self.calls = 0  # Save func for later call
        self.func = func
    
    def __call__(self, *args, **kwargs):  # On call to original func
        self.calls += 1
        print('call %s to %s' % (self.calls, self.func.__name__))
        return self.func(*args, **kwargs)
    
    def __get__(self, instance, owner):  # On method fetch
        def wrapper(*args, **kwargs):  # Retain both inst
            return self(instance, *args, **kwargs)  # Runs __call__
            
        return wrapper

In [181]:
class Person(object):
    def __init__(self, name, pay):
        self.name = name
        self.pay = pay
    
    @tracer
    def give_raise(self, percent):
        self.pay *= (1.0 + percent)
    
    @tracer
    def last_name(self):
        return self.name.split()[-1]

In [182]:
bob = Person('Bob Smith', 100)
bob.give_raise(0.2)

call 1 to give_raise


## Timing Calls

In [189]:
import time, sys
force = list if sys.version_info[0] == 3 else (lambda x: x)

class timer:
    def __init__(self, func):
        self.func = func
        self.alltime = 0
    
    def __call__(self, *args, **kwargs):
        start = time.clock()
        result = self.func(*args, **kwargs)
        elapsed = time.clock() - start
        self.alltime += elapsed
        print('%s: %.5f, %.5f' % (self.func.__name__, elapsed, self.alltime))
        return result

In [190]:
@timer
def listcomp(n):
    return [x * 2 for x in range(n)]

@timer
def mapcall(n):
    return force(map((lambda x: x * 2), range(n)))

In [191]:
result = listcomp(5)
listcomp(50000)
listcomp(500000)
listcomp(1000000)
print(result)
print('allTime = %s' % listcomp.alltime)  # Total time for all listcomp calls

listcomp: 0.00000, 0.00000
listcomp: 0.00000, 0.00000
listcomp: 0.04000, 0.04000
listcomp: 0.07000, 0.11000
[0, 2, 4, 6, 8]
allTime = 0.10999999999999988


In [193]:
result = mapcall(5)
mapcall(50000)
mapcall(500000)
mapcall(1000000)
print(result)
print('allTime = %s' % mapcall.alltime)  # Total time for all mapcall calls

print('\n**map/comp = %s' % round(mapcall.alltime / listcomp.alltime, 3))

mapcall: 0.00000, 0.21000
mapcall: 0.01000, 0.22000
mapcall: 0.05000, 0.27000
mapcall: 0.13000, 0.40000
[0, 2, 4, 6, 8]
allTime = 0.4000000000000008

**map/comp = 3.636


Ну и конфигурируемое декорирование

In [195]:
import time

def timer(label='', trace=True):
    class Timer:       
        def __init__(self, func):
            self.func = func
            self.alltime = 0
        def __call__(self, *args, **kwargs):
            start = time.clock()
            result = self.func(*args, **kwargs)
            elapsed = time.clock() - start
            self.alltime += elapsed
            if trace:
                values = (label, self.func.__name__, elapsed, self.alltime)
                print('%s %s: %.5f, %.5f' % values)
            return result
    return Timer

In [202]:
@timer(label='===> ', trace=True)
def listcomp(n):
    return [x * 2 for x in range(n)]

@timer(label='---> ')
def mapcall(n):
    return list(map((lambda x: x * 2), range(n)))

In [203]:
for func in (listcomp, mapcall):
    func(5) # Time for this call, all calls, return value
    func(50000)
    func(500000)
    func(1000000)
    print('allTime = %s\n' % func.alltime) # Total time for all calls

===>  listcomp: 0.00000, 0.00000
===>  listcomp: 0.01000, 0.01000
===>  listcomp: 0.02000, 0.03000
===>  listcomp: 0.06000, 0.09000
allTime = 0.08999999999999897

--->  mapcall: 0.00000, 0.00000
--->  mapcall: 0.00000, 0.00000
--->  mapcall: 0.05000, 0.05000
--->  mapcall: 0.13000, 0.18000
allTime = 0.17999999999999972



In [204]:
print('\n**map/comp = %s' % round(mapcall.alltime / listcomp.alltime, 3))


**map/comp = 2.0


# Программирование декораторов классов

## Singleton classes

In [206]:
instances = {}

def singleton(klass):
    def on_call(*args, **kwargs):
        if klass not in instances:
            instances[klass] = klass(*args, **kwargs)
        return instances[klass]
    return on_call

In [207]:
@singleton
class Person:
    def __init__(self, name, hours, rate):
        self.name = name
        self.hours = hours
        self.rate = rate
    def pay(self):
        return self.hours * self.rate

In [208]:
@singleton
class Spam:
    def __init__(self, val):
        self.attr = val

In [209]:
bob = Person('Bob', 40, 10)
print(bob.name, bob.pay())

Bob 400


In [210]:
sue = Person('Sue', 50, 20)
print(sue.name, sue.pay())

Bob 400


In [217]:
x = Spam(val=42)
y = Spam(99)
x.attr == y.attr == 42

True

In [212]:
instances

{__main__.Person: <__main__.Person at 0x7fbe6ee64f28>,
 __main__.Spam: <__main__.Spam at 0x7fbe6ee64a90>}

Using **nonlocal**

In [213]:
def singleton(klass):
    instance = None
    def wrapper(*args, **kwargs):
        nonlocal instance
        if instance == None:
            instance = klass(*args, **kwargs)
        return instance
    return wrapper

In [214]:
@singleton
class Spam:
    def __init__(self, val):
        self.attr = val

In [216]:
x = Spam(val=42)
y = Spam(99)
x.attr == y.attr == 42

True

Or using attribute

In [219]:
# 3.X and 2.X: func attrs, classes (alternative codings)
def singleton(aClass):
    def onCall(*args, **kwargs):
        if onCall.instance == None:
            onCall.instance = aClass(*args, **kwargs)
        return onCall.instance
    onCall.instance = None
    return onCall

class singleton:
    def __init__(self, aClass): # On @ decoration
        self.aClass = aClass
        self.instance = None
    def __call__(self, *args, **kwargs): # On instance creation
        if self.instance == None:
            self.instance = self.aClass(*args, **kwargs) # One instance per class
        return self.instance

## Tracing object interfaces

In [223]:
class Wrapper:
    def __init__(self, obj):
        self.wrapped = obj
    def __getattr__(self, attr):
        print('Trace:', attr)
        return getattr(self.wrapped, attr)

In [224]:
x = Wrapper([1, 2, 3])

In [225]:
x.append(4)

Trace: append


In [226]:
x.wrapped

[1, 2, 3, 4]

### Tracing with class decorator

In [229]:
def Tracer(klass):
    class Wrapper:
        def __init__(self, *args, **kwargs):
            self.fetches = 0
            self.wrapped = klass(*args, **kwargs)
        def __getattr__(self, attr):
            print('Trace:', attr)
            self.fetches += 1
            return getattr(self.wrapped, attr)
    return Wrapper

In [230]:
@Tracer
class Spam:
    def display(self):
        print('Spam!' * 8)

In [231]:
food = Spam()
food.display()
print(food.fetches)

Trace: display
Spam!Spam!Spam!Spam!Spam!Spam!Spam!Spam!
1
